In [ ]:

import torch
print(torch.cuda.is_available())
print(torch.cuda.get_device_capability())

True
(6, 0)


In [ ]:
%tensorflow_version 2.x

In [ ]:
import tensorflow as tf
print(tf.__version__)

2.1.0-rc1


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
!cp "/content/drive/My Drive/koulu_honmia/kandi19/IDA.zip" /content/

In [ ]:
!cp "/content/drive/My Drive/koulu_honmia/kandi19/benthic/loadbm.py" /content/

In [ ]:
%%capture
!unzip IDA.zip

In [ ]:
import numpy as np
import os
import ntpath
import platform
import matplotlib.pyplot as plt
from loadbm import create_df, create_tf_dataset, prepare_for_training


datapath = 'IDA/Separate lists with numbering/Machine learning splits'
img_path = 'IDA/Images/'

split = 1

train_fname = 'train'+str(split)+'.txt'
test_fname = 'test'+str(split)+'.txt'
val_fname = 'val'+str(split)+'.txt'

part_dat = False

df_train = create_df(os.path.join(datapath, train_fname),
                     img_path,
                     partial_dataset=part_dat,
                     seed=123)

df_test = create_df(os.path.join(datapath, test_fname),
                     img_path,
                     partial_dataset=part_dat,
                     seed=123)

df_val = create_df(os.path.join(datapath, val_fname),
                     img_path,
                     partial_dataset=part_dat,
                     seed=123)

from sklearn.utils import shuffle

df_train = shuffle(df_train)
df_test = shuffle(df_test)
df_val = shuffle(df_val)

df_train.head()

,path,label
25806,IDA/Images/Baetis_rho118/0-Baetis_rho118.1.201...,4
110987,IDA/Images/Hydraena150/1-Hydraena150.1.2016-10...,12
294494,IDA/Images/Simuliidae504/1-Simuliidae504.1.201...,37
53077,IDA/Images/Elmis_aen210/0-Elmis_aen210.1.2016-...,8
54751,IDA/Images/Elmis_aen254/0-Elmis_aen254.1.2016-...,8


In [ ]:
AUTOTUNE = tf.data.experimental.AUTOTUNE
IMSIZE = (224,224,3)
BATCH_SIZE = 32

train_ds = create_tf_dataset(df_train, imsize=IMSIZE, onehot=True)

val_ds = create_tf_dataset(df_val, imsize=IMSIZE, onehot=True)


train_ds = prepare_for_training(train_ds, 
                                shuffle_buffer_size=1000,
                                batch_size=BATCH_SIZE)

val_ds = prepare_for_training(val_ds, 
                              shuffle_buffer_size=1000,
                              batch_size=BATCH_SIZE)

for image, label in train_ds.take(5):
    print(image.shape)
    print(label.shape)


(32, 224, 224, 3)
(32, 39)
(32, 224, 224, 3)
(32, 39)
(32, 224, 224, 3)
(32, 39)
(32, 224, 224, 3)
(32, 39)
(32, 224, 224, 3)
(32, 39)


In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D

import tensorflow_addons as tfa


def get_pretrained(imsize=(224, 224, 3), classes=39):
        base_model = InceptionV3(input_shape = imsize, 
                                 weights='imagenet', 
                                 include_top=False)
        
        base_model.trainable = True
        
        x = base_model.output
        x = GlobalAveragePooling2D()(x)
        x = Dense(256, activation='relu')(x)
        predictions = Dense(classes, activation='softmax')(x)
        
        model = Model(inputs=base_model.input, outputs=predictions)
        
        return model

model = get_pretrained()
model.compile(optimizer = 'adam', loss = tfa.losses.SigmoidFocalCrossEntropy(),
                  metrics=['accuracy'])


In [ ]:
from tensorflow.keras.callbacks import CSVLogger
import datetime

csv_logger = CSVLogger('22-01-2020_colab.log',append=True)

tr_steps = len(df_train)//BATCH_SIZE
val_steps = len(df_val)//BATCH_SIZE

model.fit(train_ds, 
          validation_data= val_ds, 
          steps_per_epoch= tr_steps, 
          epochs = 10,
          validation_steps = val_steps,
          callbacks=[csv_logger])


Train for 10043 steps, validate for 1434 steps
Epoch 1/10
10043/10043 [==============================] - 1628s 162ms/step - loss: 0.0027 - accuracy: 0.8350 - val_loss: 0.0047 - val_accuracy: 0.7679
Epoch 2/10
10043/10043 [==============================] - 1610s 160ms/step - loss: 8.1580e-04 - accuracy: 0.9522 - val_loss: 0.0044 - val_accuracy: 0.8262
Epoch 3/10
10043/10043 [==============================] - 1601s 159ms/step - loss: 4.8807e-04 - accuracy: 0.9730 - val_loss: 0.0033 - val_accuracy: 0.8636
Epoch 4/10
10043/10043 [==============================] - 1596s 159ms/step - loss: 3.4619e-04 - accuracy: 0.9812 - val_loss: 0.0030 - val_accuracy: 0.8767
Epoch 5/10
10043/10043 [==============================] - 1593s 159ms/step - loss: 2.7985e-04 - accuracy: 0.9852 - val_loss: 0.0032 - val_accuracy: 0.8654
Epoch 6/10
10043/10043 [==============================] - 1612s 160ms/step - loss: 2.2327e-04 - accuracy: 0.9882 - val_loss: 0.0034 - val_accuracy: 0.8669
Epoch 7/10
10043/10043 [===

In [ ]:
from tensorflow.keras.optimizers import Adam
adam = Adam(learning_rate=0.0001) #original 0.001

from tensorflow.keras.callbacks import LearningRateScheduler
def scheduler(epoch):
    return 0.0001 * tf.math.exp(0.1 * (-epoch))

lr_cb = LearningRateScheduler(scheduler)

model.compile(optimizer = adam, loss = 'categorical_crossentropy',
                  metrics=['accuracy'])

model.fit(train_ds, 
          validation_data= val_ds, 
          steps_per_epoch= tr_steps, 
          epochs = 5,
          validation_steps = val_steps,
          callbacks=[csv_logger, lr_cb])

Train for 10043 steps, validate for 1434 steps
Epoch 1/5
10043/10043 [==============================] - 1609s 160ms/step - loss: 0.0039 - accuracy: 0.9993 - val_loss: 0.3579 - val_accuracy: 0.9247
Epoch 2/5
10043/10043 [==============================] - 1596s 159ms/step - loss: 9.9305e-04 - accuracy: 0.9998 - val_loss: 0.3850 - val_accuracy: 0.9261
Epoch 3/5
10043/10043 [==============================] - 1600s 159ms/step - loss: 5.4951e-04 - accuracy: 0.9999 - val_loss: 0.3917 - val_accuracy: 0.9297
Epoch 4/5
10043/10043 [==============================] - 1606s 160ms/step - loss: 4.6909e-04 - accuracy: 0.9999 - val_loss: 0.4272 - val_accuracy: 0.9288
Epoch 5/5
10043/10043 [==============================] - 1606s 160ms/step - loss: 3.1874e-04 - accuracy: 0.9999 - val_loss: 0.4205 - val_accuracy: 0.9273


In [ ]:
model.save('22-01-2020_cont_colab.h5')

In [ ]:
!cp '22-01-2020_cont_colab.h5' "/content/drive/My Drive/koulu_honmia/kandi19/"

Evaluation:

In [ ]:
test_ds = create_tf_dataset(df_test, imsize=IMSIZE, onehot=True)
test_ds = test_ds.batch(BATCH_SIZE)

In [ ]:
!cp "/content/drive/My Drive/koulu_honmia/kandi19/benthic/combine_insects.py" /content/

cp: cannot stat '/content/drive/My Drive/koulu_honmia/kandi19/benthic/combine_insects.py': No such file or directory


In [ ]:
preds = model.predict(test_ds, verbose=True)
yhat = np.argmax(preds,axis=1)+1
y_test = df_test['label']

acc = np.sum(yhat==y_test)/len(y_test)
print('Image accuracy: {:.4f}'.format(acc))

np.save('22-01-2020_cont_colab.npy', preds)

In [ ]:
!cp '22-01-2020_cont_colab.npy' "/content/drive/My Drive/koulu_honmia/kandi19/"

In [ ]:
#%% Insect combine
from combine_insects import add_insect_class, add_yhat

df_test_preds = add_insect_class(df_test)

# adding predictions to dataframe for insect-wise prediction
df_test_preds = add_yhat(df_test_preds,yhat)

dfg = df_test_preds.groupby(['label','insect'],as_index=False)['pred'].agg(lambda x:x.value_counts().index[0])

acc_g = np.sum(dfg['pred']==dfg['label'])/len(dfg)
print('Aggregate accuracy: {:.4f}'.format(acc_g))